## Voting Classifiers
* *Hard voting:* majority-voting classifier
* *Soft voting* predicts the class with the highest probability averaged over all the individual classifiers. It often achieved higher performance than hard voting because it gives more weight to highly confident votes.
* For classifiers that does not generate probability such as SVC, check [Platt Scaling] (https://en.wikipedia.org/wiki/Platt_scaling). Basically it snaps a logistic function on top of the SVM decision function, and calibrate the parameters using cross-validation.
* **Why does ensembling work?** *Law of large numbers*. think of tossing a biased coin many times, the more you tosse it, the more likely you are going to find the head. Similarly, ensemble of *weak leaners* can still be a *strong learner (achieving high accuracy)* provided there are a sufficient number of weak learners and they are sufficiently diverse.


## Bagging and Pasting
* *Bagging*: sampling *with* replacement. *Pasting*: *without* replacement.
* [Bagging VS Pasting](https://stats.stackexchange.com/questions/219193/when-should-the-pasting-ensemble-method-be-used-instead-of-bagging), basically when size of the dataset is small, bagging is always the choice, pasting might be preferrable with large sample size and on external validations.
* In Scikit-Learn, it is selected by the hyperparameter **bootstrap** (*=True* for bagging, *=False* for pasting.)

### Out-of-Bag Evaluation
* in Scikit-Learn, set **oob_score=True** to request automatic oob evaluations after training, which could be used as an estimate for testing set performance.

## Random Forests

* Sampling both training instances and features is called *Random Patches*; keeping all training instances but sampling features is called *Random Subspaces*.
* RF searches for the best feature among a random subset of features when growing trees in order to introduce extra randomness. It trades a higher bias for a lower variance, generally yielding an overall better model. 

### Extra-Trees
* On top of RF, also using random thresholds for each feature rather than searching for the best possible thresholds like regular Decision Trees do. Called *Extremely Randomized Trees* ensemble.
* Much faster to train.

### Feature Importance
* Important features are likely to appear closer to the root, so it is possible to estimate a feature's importance by computing the average depth at which it appears across all trees in the forest.
* [Gini importance](https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm#giniimp): "Every time a split of a node is made on feature $m$ the gini impurity criterion for the two children nodes is less than the parent node. Adding up the gini decreases for each individual feature over all trees in the forest gives a fast feature importance that is often very consistent with the permutation importance measure."

## Boosting

### AdaBoost

### Gradient Boosting

## Stacking

## Resources

* Detailed [implementation](https://github.com/ageron/handson-ml/blob/master/07_ensemble_learning_and_random_forests.ipynb) of this chapter by [Aurélien Geron](https://github.com/ageron)
* [Bagging VS Pasting](https://stats.stackexchange.com/questions/219193/when-should-the-pasting-ensemble-method-be-used-instead-of-bagging)
* [Gini importance](https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm#giniimp)